In [11]:
import pandas as pd
import numpy as np
from glob import glob
import os
import math
import wandb

from PIL import Image
from tqdm.notebook import tqdm

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader 
from torchvision import transforms

from sklearn.utils import class_weight, shuffle
from sklearn.metrics import f1_score, fbeta_score, accuracy_score
from sklearn.model_selection import train_test_split

In [23]:
wandb.init()

accuracy,▁
f1_score,▁
accuracy,0.46772
f1_score,0.22928


In [24]:
wandb.run.name = 'resnet-run-test'

In [25]:
train_dir = '/opt/ml/input/data/train'
train_df = pd.read_csv('/opt/ml/input/data/train/train_data_total.csv')
train_df

,Unnamed: 0,id,gender,img_path,age,age_group,mask,class
0,0,000001,female,/opt/ml/input/data/train/images/000001_female_...,45,>= 30 and < 60,mask,4
1,1,000001,female,/opt/ml/input/data/train/images/000001_female_...,45,>= 30 and < 60,mask,4
2,2,000001,female,/opt/ml/input/data/train/images/000001_female_...,45,>= 30 and < 60,normal,16
3,3,000001,female,/opt/ml/input/data/train/images/000001_female_...,45,>= 30 and < 60,incorrect_mask,10
4,4,000001,female,/opt/ml/input/data/train/images/000001_female_...,45,>= 30 and < 60,mask,4
...,...,...,...,...,...,...,...,...
18895,18895,006959,male,/opt/ml/input/data/train/images/006959_male_As...,19,< 30,normal,12
18896,18896,006959,male,/opt/ml/input/data/train/images/006959_male_As...,19,< 30,incorrect_mask,6
18897,18897,006959,male,/opt/ml/input/data/train/images/006959_male_As...,19,< 30,mask,0
18898,18898,006959,male,/opt/ml/input/data/train/images/006959_male_As...,19,< 30,mask,0


In [101]:
class CustomDataset(Dataset):
    def __init__(self, df_data, train=True):
        self.data = df_data
        self.label = self.data['class'].tolist()
        self.img_path = self.data['img_path'].tolist()
        
        self.transform = transforms.ToTensor()

        self.train = train
    
    def __len__(self):
        return len(self.label)
    
    def __getitem__(self, index):
        img = Image.open(self.img_path[index])
        img = img.resize((32, 32))
        img = np.asarray(img)/255
        
        img_tensor = self.transform(img)
        
        if self.train:
            return (self.label[index], img_tensor)
        else:
            return img_tnesor
        
        
        return img

In [102]:
SEED = 42
TEST_SIZE =0.15
train_set, valid_set = train_test_split(train_df, test_size=TEST_SIZE, random_state=SEED)

In [103]:
print(train_set.shape, valid_set.shape)

(16065, 8) (2835, 8)


In [104]:
train_dataset = CustomDataset(train_set)
valid_dataset = CustomDataset(valid_set)

In [105]:
pre_model = torchvision.models.resnet18(pretrained=True)

In [106]:
BATCH_SIZE = 7
train_dataset_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=False)
valid_dataset_loader = torch.utils.data.DataLoader(dataset=valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

dataloaders = {
    "train" : train_dataset_loader,
    "test" : valid_dataset_loader
}

class_num = 18
pre_model.fc = torch.nn.Linear(in_features=512, out_features=class_num, bias=True)
torch.nn.init.xavier_uniform_(pre_model.fc.weight)

stdv = 1. / math.sqrt(pre_model.fc.weight.size(1))
pre_model.fc.bias.data.uniform_(-stdv, stdv)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pre_model.to(device)

LEARNING_RATE = 0.0001
NUM_EPOCH = 30

In [ ]:
from torch.autograd import Variable

criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(pre_model.parameters(), lr=LEARNING_RATE, momentum=0.9)
optimizer = torch.optim.Adam(pre_model.parameters(), lr=LEARNING_RATE)

for epoch in range(NUM_EPOCH):
    for i, (labels, images) in enumerate(train_dataset_loader):
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        
        optimizer.zero_grad()
        
        outputs = pre_model(images.float())
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        if i%200 == 0:
            print(f"Eopch: {epoch} | Batch ID: {i} | Loss: {loss.data}")
    print()

print("done!")

Eopch: 0 | Batch ID: 0 | Loss: 4.062315464019775
Eopch: 0 | Batch ID: 200 | Loss: 3.099478244781494
Eopch: 0 | Batch ID: 400 | Loss: 2.831279754638672
Eopch: 0 | Batch ID: 600 | Loss: 1.8681915998458862
Eopch: 0 | Batch ID: 800 | Loss: 1.9934724569320679
Eopch: 0 | Batch ID: 1000 | Loss: 2.6563308238983154
Eopch: 0 | Batch ID: 1200 | Loss: 1.5240802764892578
Eopch: 0 | Batch ID: 1400 | Loss: 1.2494004964828491
Eopch: 0 | Batch ID: 1600 | Loss: 3.1018223762512207
Eopch: 0 | Batch ID: 1800 | Loss: 1.2552002668380737
Eopch: 0 | Batch ID: 2000 | Loss: 2.4217803478240967
Eopch: 0 | Batch ID: 2200 | Loss: 2.338005304336548

Eopch: 1 | Batch ID: 0 | Loss: 2.3525519371032715
Eopch: 1 | Batch ID: 200 | Loss: 1.0701574087142944
Eopch: 1 | Batch ID: 400 | Loss: 1.31036376953125
Eopch: 1 | Batch ID: 600 | Loss: 0.9753339886665344
Eopch: 1 | Batch ID: 800 | Loss: 1.192132830619812
Eopch: 1 | Batch ID: 1000 | Loss: 2.2136075496673584
Eopch: 1 | Batch ID: 1200 | Loss: 1.4142581224441528
Eopch: 1 | Ba

In [ ]:
targets = []
all_predictions = []

for labels, images in valid_dataset_loader:
    with torch.no_grad():
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        
        pred = pre_model(images.float())
        pred = pred.argmax(dim=-1)
        
        targets.extend(labels.cpu().numpy())
        
        all_predictions.extend(pred.cpu().numpy())

        
accuracy_val = accuracy_score(targets, all_predictions)
f1_score_val = np.mean(f1_score(targets, all_predictions, average=None))
print(f"Accuracy: {accuracy_val}")
print(f"F1 Score: {f1_score_val}")

wandb.log({'accuracy': accuracy_val, 
           'f1_score': f1_score_val,
           'BATCH_SIZE': BATCH_SIZE,
           'NUM_EPOCH': NUM_EPOCH,
           'LEARNING_RATE': LEARNING_RATE,
           'SEED': SEED,
           'TEST_SIZE':TEST_SIZE})